<a href="https://colab.research.google.com/github/toppac/ACbook/blob/master/DreamBooth_Stable_Diffusion_(Adapted_to_A_Novel_Model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is designed to run smoothly on a Tesla T4 GPU.

## Possible Issues

If you encounter weird errors, that's probably because Colab is messing with your HTTP connection.

Out of VRAM error won't happen if you are using the default settings and Tesla T4.

In [ ]:
#@title Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [ ]:
#@title Install Requirements
!apt install -y -qq aria2 > /dev/null

# !curl -LO https://github.com/huggingface/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
# use memory-optim version
!git clone https://github.com/CCRcmcpe/diffusers.git

TRAINER = "diffusers/examples/dreambooth/train_dreambooth.py"
CONVERTER = "diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py"
BACK_CONVERTER = "diffusers/scripts/convert_diffusers_to_original_stable_diffusion.py"

!pip install -U pip
%pip install -q ./diffusers
%pip install -q -U --pre triton
%pip install -q accelerate==0.12.0 transformers==4.24.0 ftfy==6.1.1 bitsandbytes==0.35.4 omegaconf==2.2.3 einops==0.5.0 pytorch-lightning==1.7.7 gradio

from subprocess import getoutput
from IPython.display import HTML
from IPython.display import clear_output
import time
import os

s = getoutput('nvidia-smi')
if 'T4' in s:
  gpu = 'T4'
elif 'P100' in s:
  gpu = 'P100'
elif 'V100' in s:
  gpu = 'V100'
elif 'A100' in s:
  gpu = 'A100'

while True:
    try: 
        gpu=='T4'or gpu=='P100'or gpu=='V100'or gpu=='A100'
        break
    except:
        pass
    print('It seems that your GPU is not supported at the moment')
    time.sleep(5)

if (gpu=='T4'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/T4/xformers-0.0.13.dev0-py3-none-any.whl
  
elif (gpu=='P100'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='V100'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl

elif (gpu=='A100'):
  %pip install https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl  

In [ ]:
#@title Login to wandb to watch training process (Optional, keep key empty if you want to skip)
WANDB_KEY = "" #@param {type:"string"}
if WANDB_KEY != "":
  %pip install wandb
  !wandb login $WANDB_KEY

In [ ]:
#@title Download model and convert it to HF format.

#@markdown URL of the model (default: animefull-latest).
URL = "https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar" #@param {type:"string"}
#@markdown VAE weights. You may leave it empty.
VAE_URL = "https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animevae.pt" #@param {type:"string"}

#@markdown Model download path.
SRC_PATH = "/content/model-sd" #@param {type:"string"}

#@markdown SD to HF format convert output path.
MODEL_NAME = "/content/model-hf" #@param {type:"string"}

#@markdown Download example training data (nahida)?
download_example_traindata = True #@param {type:"boolean"}

!echo "Downloading model..."
!mkdir -p $SRC_PATH
%pushd $SRC_PATH

!aria2c --summary-interval=5 -x 6 --allow-overwrite=true -o data.tar $URL

if VAE_URL != "":
  !aria2c --summary-interval=5 -x 6 --allow-overwrite=true -o vae.pt $VAE_URL
  vae_arg = f"--vae_path {SRC_PATH}/vae.pt"

!tar xf data.tar 
!rm data.tar

%popd
!echo "Done"

!echo "Converting model..."

!python $CONVERTER --checkpoint_path $SRC_PATH/model.ckpt --original_config_file $SRC_PATH/config.yaml $vae_arg --dump_path $MODEL_NAME --scheduler_type ddim

!echo "Done"

!rm -r $SRC_PATH

if download_example_traindata and not os.path.exists('instance-images'):
  !mkdir instance-images
  !aria2c --summary-interval=5 -x 6 --allow-overwrite=true https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/train-nahida.tar
  !tar x -C instance-images -f train-nahida.tar
  !rm train-nahida.tar
  !rm -r instance-images/._train*


## Instance Prompt and Class Prompt

What your training set is about|Instance prompt must contain|Class prompt should describe
-|-|-
A object/person|`[V]`|The object's type and/or characteristics
A artist's style|`by [V]`|The common characteristics of the training set

Where:
* `[V]` is a *token* in CLIP's [vocabulary](https://huggingface.co/openai/clip-vit-large-patch14/raw/main/vocab.json) which is not meaningful to the model. `sks` is a great example.

A common pitfall: like if you are training about a specific person with name `[N]`, you should NOT use `[N]` as `[V]`. Names have high chance of being separated (tokenized) to multiple tokens, which is possibly hazardous.

Finally `[V]` will carry the new information learned by the model.

### Examples

Training about a female character:
* Instance prompt: `sks 1girl`
* Class prompt: `1girl`

Training about hatsune miku (don't do this btw, model already knows):
* Instance prompt: `masterpiece, best quality, sks 1girl, aqua eyes, aqua hair`
* Class prompt: `masterpiece, best quality, 1girl, aqua eyes, aqua hair`

Training about an artist's style on drawing female characters:
* Instance prompt: `1girl, by sks`
* Class prompt: `1girl`

In [ ]:
#@title Settings

#@markdown ## Train set

INSTANCE_PROMPT = "masterpiece, best quality, sks 1girl" #@param {type:"string"}

#@markdown Training set path (images of the subject).
INSTANCE_DIR = "/content/instance-images" #@param {type:"string"}

#@markdown ## Class set
CLASS_PROMPT = "masterpiece, best quality, 1girl" #@param {type:"string"}
CLASS_NEGATIVE_PROMPT = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry" #@param {type:"string"}
CLASS_DIR = "/content/class-images" #@param {type:"string"}
NUM_CLASS_IMAGES = 100 #@param {type:"number"}

#@markdown ## Previewing
#@markdown Prompt for saving samples.
SAVE_SAMPLE_PROMPT = "masterpiece, best quality, sks 1girl, looking at viewer" #@param {type:"string"}
SAVE_SAMPLE_NEGATIVE_PROMPT = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry" #@param {type:"string"}

#@markdown ## Output
#@markdown If model weights should be saved directly to google drive (takes around 2-3 GB).
save_to_gdrive = False #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

#@markdown Enter the directory name to save model at.

OUTPUT_DIR = "output" #@param {type:"string"}
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

## Advanced

Use the table below to choose the best flags based on your memory and speed requirements. Tested on Tesla T4 GPU.

| `fp16` | `train_batch_size` | `gradient_accumulation_steps` | `gradient_checkpointing` | `use_8bit_adam` | GB VRAM usage | Speed (it/s) |
| ---- | ------------------ | ----------------------------- | ----------------------- | --------------- | ---------- | ------------ |
| fp16 | 1                  | 1                             | TRUE                    | TRUE            | 9.92       | 0.93         |
| no   | 1                  | 1                             | TRUE                    | TRUE            | 10.08      | 0.42         |
| fp16 | 2                  | 1                             | TRUE                    | TRUE            | 10.4       | 0.66         |
| fp16 | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 1.14         |
| no   | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 0.49         |
| fp16 | 1                  | 2                             | TRUE                    | TRUE            | 11.56      | 1            |
| fp16 | 2                  | 1                             | FALSE                   | TRUE            | 13.67      | 0.82         |
| fp16 | 1                  | 2                             | FALSE                   | TRUE            | 13.7       | 0.83          |
| fp16 | 1                  | 1                             | TRUE                    | FALSE           | 15.79      | 0.77         |

If you are using a GPU better than Tesla T4, remove `--gradient_checkpointing` from the arguments to improve training speed, also consider increasing `train_batch_size` for less overhead and better regularization.

Remove `--use_8bit_adam` flag for full precision optimizer. Requires 15.79 GB with `--gradient_checkpointing` else 17.8 GB. Somewhat unreasonable to do, but if you are encountering issues with the reduced precision, oh well.

### Multiple Concepts

You can set up a `concepts_list.json` like:

```json
[
    {
        "instance_prompt":      "photo of a woman wearing sks dress",
        "class_prompt":         "photo of a woman wearing dress",
        "instance_data_dir":    "data/wrap_dress",
        "class_data_dir":       "data/dress"
    },
    {
        "instance_prompt":      "photo of sks woman",
        "class_prompt":         "photo of a woman",
        "instance_data_dir":    "data/woman1",
        "class_data_dir":       "data/woman_class"
    }
]
```

And use it with `--concepts_list concepts_list.json`. Can let model learn multiple concepts at the same time. Currently not compatible with Variable Prompts.

### Variable Prompts

For each image (`[X].png` / `[X].jpg`) in data set, put an additional `[X].txt` containing corresponding prompt with it. Then set `READ_PROMPT_FROM_TXT`. Both train set and class set supports this.

Prompt read from txt `[PX]` will be inserted to the prompt you set `[P]` in train args. By default, it is inserted like `[PX] [P]`.

With Variable Prompts enabled and prior preservation loss disabled (`PRIOR_PRESERVATION`), the training process is effectively an equivalent to standard finetuning.

### Aspect Ratio Bucketing

Used by NovelAI when they train their model. In a nutshell, it sets variable training resolution, eliminating the need of cropping dataset manually to 1:1 while still preserving information well. Brings better result especially when generating images with aspect ratio ≠ 1.

Cost is it slows down training and requires slightly more VRAM. Tested using it with batch size = 2 on Tesla T4 is fine.

### Optimizer

The default is int8 AdamW. Works well. If you want to use SGDM, prepare to get into troubles like model does not give much different results after many steps.

### Storage Issue

To save Colab from out of storage, by default we save unet weights using FP16 (`--save_unet_half`).

If you enabled wandb, you can add `--wandb_artifact` to upload weights to wandb. Optionally, `--rm_after_wandb_saved` can let weights be removed after uploading. (Because Colab somewhat actively mess with connections, this is disabled by default.)


In [ ]:
#@title Advanced Parameters
MAX_TRAIN_STEPS = 2000 #@param {type:"number"}
SAVE_INTERVAL = 500 #@param {type:"number"}
SEED = 114514 #@param {type:"number"}
#@markdown ## Data Processing
RESOLUTION = 512 #@param {type:"slider", min:64, max:2048, step:28}
ASPECT_RATIO_BUCKETING = False #@param {type:"boolean"}
READ_PROMPT_FROM_TXT = "no" #@param ["no", "instance", "class", "both"] {allow-input: false}
#@markdown ## Forward Pass
TRAIN_BATCH_SIZE = 1 #@param {type:"slider", min:1, max:10, step:1}
GRADIENT_ACCUMULATION_STEPS = 1 #@param {type:"slider", min:1, max:10, step:1}
CLIP_SKIP = 2 #@param {type:"slider", min:1, max:6, step:1}
MIXED_PRECISION = "fp16" #@param ["no", "fp16", "bf16"] {allow-input: false}
#@markdown ## Optimizer / Backward Pass
OPTIMIZER = "adamw_8bit" #@param ["adamw", "adamw_8bit", "adamw_ds", "sgdm", "sgdm_8bit"] {allow-input: false}
LEARNING_RATE = 5e-6 #@param {type:"number"}
LR_SCHEDULER = "cosine_with_restarts"  #@param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"] {allow-input: false}
LR_WARMUP_STEPS = 100 #@param {type:"number"}
LR_CYCLES = 1 #@param {type:"number"}
LAST_EPOCH = -1 #@param {type:"number"}
SCALE_LR = True #@param {type:"boolean"}
PRIOR_PRESERVATION = True #@param {type:"boolean"}
PRIOR_LOSS_WEIGHT = 1 #@param {type:"slider", min:0, max:1, step:0.01}
#@markdown ## Inference (Class Set Generation / Sample Images Generation)
INFER_STEPS = 28 #@param {type:"integer"}
GUIDANCE_SCALE = 11 #@param {type:"integer"}
SAMPLE_N = 4  #@param {type:"integer"}
INFER_BATCH_SIZE = 2 #@param {type:"slider", min:1, max:10, step:1}

In [ ]:
#@title HF Accelerate Config
%%bash

mkdir -p ~/.cache/huggingface/accelerate

cat > ~/.cache/huggingface/accelerate/default_config.yaml <<- EOM
compute_environment: LOCAL_MACHINE
deepspeed_config: {}
distributed_type: 'NO'
downcast_bf16: 'no'
fsdp_config: {}
machine_rank: 0
main_process_ip: null
main_process_port: null
main_training_function: main
mixed_precision: fp16
num_machines: 1
num_processes: 1
use_cpu: false
EOM

In [ ]:
%cd /content
!mkdir -p $OUTPUT_DIR

wandb_arg = "--wandb" if WANDB_KEY != "" else ""
scale_lr_arg = "--scale_lr" if SCALE_LR else ""
ppl_arg = f"--with_prior_preservation --prior_loss_weight={PRIOR_LOSS_WEIGHT}" if PRIOR_PRESERVATION else ""
read_prompt_arg = f"--read_prompt_from_txt {READ_PROMPT_FROM_TXT}" if READ_PROMPT_FROM_TXT != "no" else ""
arb_arg = "--use_aspect_ratio_bucket --debug_arb" if ASPECT_RATIO_BUCKETING else ""


!accelerate launch $TRAINER \
  --instance_data_dir "{INSTANCE_DIR}" \
  --instance_prompt "{INSTANCE_PROMPT}" \
  --pretrained_model_name_or_path "{MODEL_NAME}" \
  --pretrained_vae_name_or_path "{MODEL_NAME}/vae" \
  --output_dir "{OUTPUT_DIR}" \
  --seed=$SEED \
  --resolution=$RESOLUTION \
  --optimizer "{OPTIMIZER}" \
  --train_batch_size=$TRAIN_BATCH_SIZE \
  --learning_rate=$LEARNING_RATE \
  --lr_scheduler=$LR_SCHEDULER \
  --lr_warmup_steps=$LR_WARMUP_STEPS \
  --lr_cycles=$LR_CYCLES \
  --last_epoch=$LAST_EPOCH \
  --max_train_steps=$MAX_TRAIN_STEPS \
  --save_interval=$SAVE_INTERVAL \
  --class_data_dir "{CLASS_DIR}" \
  --class_prompt "{CLASS_PROMPT}" --class_negative_prompt "{CLASS_NEGATIVE_PROMPT}" \
  --num_class_images=$NUM_CLASS_IMAGES \
  --save_sample_prompt "{SAVE_SAMPLE_PROMPT}" --save_sample_negative_prompt "{SAVE_SAMPLE_NEGATIVE_PROMPT}" \
  --n_save_sample=$SAMPLE_N \
  --infer_batch_size=$INFER_BATCH_SIZE \
  --infer_steps=$INFER_STEPS \
  --guidance_scale=$GUIDANCE_SCALE \
  --gradient_accumulation_steps=$GRADIENT_ACCUMULATION_STEPS \
  --gradient_checkpointing \
  --save_unet_half \
  --mixed_precision "{MIXED_PRECISION}" \
  --clip_skip=$CLIP_SKIP \
  $wandb_arg $scale_lr_arg $ppl_arg $read_prompt_arg $arb_arg

# disabled: --not_cache_latents 

## Convert weights to ckpt to use in web UIs like AUTOMATIC1111.

In [ ]:
#@markdown Which step number to use.
use_checkpoint = '2000' #@param {type:"string"}
#@markdown Id of which run to use (empty = latest run).
run_id = '' #@param {type:"string"}

if not run_id:
  runs = [d for d in Path(OUTPUT_DIR).iterdir() if d.is_dir()]
  runs.sort(lambda d: d.stat().st_ctime, reverse=True)
  run_id = runs[0].name

ckpt_path = f'{OUTPUT_DIR}/{run_id}/{use_checkpoint}/model.ckpt'

# You can add --vae and --text_encoder if you want.
!python "{BACK_CONVERTER}" --model_path "{OUTPUT_DIR}/{run_id}/{use_checkpoint}" --checkpoint_path $ckpt_path \
  --unet_dtype fp16

print(f"[*] Converted ckpt saved at {ckpt_path}")

## Inference

In [ ]:
import torch
import os
from torch import autocast
from diffusers import StableDiffusionPipeline
from IPython.display import display

#@markdown Run Gradio UI for generating images.
use_checkpoint = '2000' #@param {type:"string"}
model_path = os.path.join(OUTPUT_DIR, use_checkpoint)

pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
g_cuda = None


import gradio as gr

def inference(prompt, negative_prompt, num_samples, height=512, width=512, num_inference_steps=50, guidance_scale=7.5):
    with torch.autocast("cuda"), torch.inference_mode():
        return pipe(
                prompt, height=int(height), width=int(width),
                negative_prompt=negative_prompt,
                num_images_per_prompt=int(num_samples),
                num_inference_steps=int(num_inference_steps), guidance_scale=guidance_scale,
                generator=g_cuda
            ).images

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="photo of sks guy, digital painting")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="")
            run = gr.Button(value="Generate")
            with gr.Row():
                num_samples = gr.Number(label="Number of Samples", value=4)
                guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
            with gr.Row():
                height = gr.Number(label="Height", value=512)
                width = gr.Number(label="Width", value=512)
            num_inference_steps = gr.Slider(label="Steps", value=50)
        with gr.Column():
            gallery = gr.Gallery()

    run.click(inference, inputs=[prompt, negative_prompt, num_samples, height, width, num_inference_steps, guidance_scale], outputs=gallery)

demo.launch(debug=True)